In [1]:
import pickle

import pandas as pd
from helpers import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np

from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

/home/vecht499/.local/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
from sklearn.metrics import confusion_matrix

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == labels) / predictions.shape[0])

## Load dataset

In [3]:
with open('../data/prepared_data.pickle', 'rb') as data:
    data_dic = pickle.load(data)

In [4]:
train = data_dic['sample_df']['train']
test  = data_dic['sample_df']['test']
valid = data_dic['sample_df']['valid']

## Baseline model

In [31]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
vectorizer = TfidfVectorizer( max_features = 20000,
                              ngram_range = ( 2, 3),
                              sublinear_tf = True,
                              stop_words='english',
                              tokenizer=LemmaTokenizer())

tfidf = vectorizer.fit_transform(train["text"].tolist())
feature_names = vectorizer.get_feature_names()

In [ ]:
train_data_features=tfidf.toarray()
forest = RandomForestClassifier(n_estimators = 50) 
forest = forest.fit(train_data_features, train["cat1_num"])

In [ ]:
test_data_features = vectorizer.transform(test["text"].tolist()).toarray()
result = forest.predict(test_data_features)